In [6]:
# This is where we will do the initial data cleaning for the dataset.

import pandas as pd

# Read the CSV file into a DataFrame
filepath = '/Users/andrewhlavacek/Documents/EDA_Netflix/netflix1.csv'

df = pd.read_csv(filepath)

# Display the first few rows of the Dataframe to show that it loaded correctly
#print(df.head(10))


# First we check for any missing values in the DataFrame
#print(df.isnull().sum())

# As we can see there are no null values to worry about, so we can continue with the cleaning
# Next we will check the data types of each column
print(df.dtypes)





     show_id     type                      title   director        country  \
6619    s272  TV Show    Brand New Cherry Flavor  Not Given  United States   
6620    s235  TV Show                  Oggy Oggy  Not Given       Pakistan   
6621    s273  TV Show  Fast & Furious Spy Racers  Not Given  United States   

     date_added  release_year rating   duration  \
6619  8/13/2021          2021  TV-MA   1 Season   
6620  8/24/2021          2021   TV-Y   1 Season   
6621  8/13/2021          2021  TV-Y7  5 Seasons   

                               listed_in  
6619  TV Dramas, TV Horror, TV Mysteries  
6620               Kids' TV, TV Comedies  
6621                            Kids' TV  
